# Import Libraries

In [1]:
import os
import random
import shutil
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.layers.experimental.preprocessing import RandomFlip, RandomRotation
from tensorflow.keras.applications import ResNet50
from keras.callbacks import ModelCheckpoint

Matplotlib created a temporary cache directory at /var/folders/86/7gxks06s07xch2nsjg8bfzhh0000gn/T/matplotlib-v5b5cjcz because the default path (/Users/ingwerludwig/.matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


### Preprocessing Data

Split data 80:20 karena dataset yang sedikit


In [2]:
#load source data
class_names = ['ayam_goreng', 'ayam_pop', 'daging_rendang', 'dendeng_batokok', 'gulai_ikan', 'gulai_tambusu', 'gulai_tunjang', 'lele_goreng', 'nasi', 'tahu_goreng', 'telur_dadar_goreng', 'telur_matasapi_goreng', 'telur_rebus_balado', 'tempe_goreng']
dataset = 'D:\\dataset - Copy'

#load data train dir 
root_dir = 'D:\\train'
train_dir = os.path.join(root_dir, 'training')
validation_dir = os.path.join(root_dir, 'validation')

def create_train_val_dirs(root_path, class_names):
    for food in class_names:
        train_food_dir = os.path.join(train_dir, food)
        validation_food_dir = os.path.join(validation_dir, food)
        if not os.path.exists(train_food_dir):
            os.makedirs(train_food_dir)
        if not os.path.exists(validation_food_dir):
            os.makedirs(validation_food_dir)

In [3]:
#cek
for rootdir, dirs, files in os.walk(root_dir):
    for subdir in dirs:
        print(os.path.join(rootdir, subdir))

D:\train\training
D:\train\validation
D:\train\training\ayam_goreng
D:\train\training\ayam_pop
D:\train\training\daging_rendang
D:\train\training\dendeng_batokok
D:\train\training\gulai_ikan
D:\train\training\gulai_tambusu
D:\train\training\gulai_tunjang
D:\train\training\lele_goreng
D:\train\training\nasi
D:\train\training\tahu_goreng
D:\train\training\telur_dadar_goreng
D:\train\training\telur_matasapi_goreng
D:\train\training\telur_rebus_balado
D:\train\training\tempe_goreng
D:\train\validation\ayam_goreng
D:\train\validation\ayam_pop
D:\train\validation\daging_rendang
D:\train\validation\dendeng_batokok
D:\train\validation\gulai_ikan
D:\train\validation\gulai_tambusu
D:\train\validation\gulai_tunjang
D:\train\validation\lele_goreng
D:\train\validation\nasi
D:\train\validation\tahu_goreng
D:\train\validation\telur_dadar_goreng
D:\train\validation\telur_matasapi_goreng
D:\train\validation\telur_rebus_balado
D:\train\validation\tempe_goreng


In [4]:
def split_data(SOURCE_DIR, TRAINING_DIR, VALIDATION_DIR, SPLIT_SIZE):
    """
    Splits the data into train and validation sets.
  
    Args:
        SOURCE_DIR (string): directory path containing the images
        TRAINING_DIR (string): directory path to be used for training
        VALIDATION_DIR (string): directory path to be used for validation
        SPLIT_SIZE (float): proportion of the dataset to be used for training
    
    Returns:
        None
    """
  
    # Get the list of image files in the source directory
    source_files = os.listdir(SOURCE_DIR)
  
    # Shuffle the list of files randomly
    random.shuffle(source_files)
  
    # Calculate the split index based on the split size
    split_index = int(len(source_files) * SPLIT_SIZE)
  
    # Split the list of files into training and validation lists
    train_files = source_files[:split_index]
    validation_files = source_files[split_index:]
  
    # Copy the files to the respective directories
    for file in train_files:
        source_file = os.path.join(SOURCE_DIR, file)
        destination_file = os.path.join(TRAINING_DIR, file)
        shutil.copyfile(source_file, destination_file)
  
    for file in validation_files:
        source_file = os.path.join(SOURCE_DIR, file)
        destination_file = os.path.join(VALIDATION_DIR, file)
        shutil.copyfile(source_file, destination_file)

In [5]:
def split_data_combine(food):
    # Define paths
    FOOD_SOURCE_DIR = "D:\\dataset - Copy\\{}\\".format(food)
    
    TRAINING_DIR = "D:\\train\\training\\"
    VALIDATION_DIR = "D:\\train\\validation\\"
    
    TRAINING_FOOD_DIR = os.path.join(TRAINING_DIR, "{}\\".format(food))
    VALIDATION_FOOD_DIR = os.path.join(VALIDATION_DIR, "{}\\".format(food))
    
    if len(os.listdir(TRAINING_FOOD_DIR)) > 0:
        for file in os.scandir(TRAINING_FOOD_DIR):
            os.remove(file.path)
            
    if len(os.listdir(VALIDATION_FOOD_DIR)) > 0:
        for file in os.scandir(VALIDATION_FOOD_DIR):
            os.remove(file.path)
    
    split_size = 0.8
    
    split_data(FOOD_SOURCE_DIR, TRAINING_FOOD_DIR, VALIDATION_FOOD_DIR, split_size)


### Image Augmentation

In [6]:
def train_val_generators(TRAINING_DIR, VALIDATION_DIR):
    
    train_datagen = ImageDataGenerator(rescale=1./255.,
                                       rotation_range=30,       # Random rotation between -30 and 30 degrees
                                       width_shift_range=0.2,   # Randomly shift the width by 20%
                                       height_shift_range=0.2,  # Randomly shift the height by 20%
                                       shear_range=0.2,         # Randomly apply shear transformation
                                       zoom_range=0.2,          # Randomly zoom in or out by 20%
                                       horizontal_flip=True,    # Randomly flip the image horizontally
                                       vertical_flip=True,      # Randomly flip the image vertically
                                       brightness_range=(0.8, 1.2),  # Randomly adjust brightness between 0.8 and 1.2
                                       fill_mode='nearest')
    
    train_generator = train_datagen.flow_from_directory(directory=TRAINING_DIR,
                                                        batch_size=32,
                                                        class_mode='categorical',
                                                        target_size=(224, 224))
    
    validation_datagen = ImageDataGenerator( rescale = 1./255.)

    validation_generator = validation_datagen.flow_from_directory(directory=VALIDATION_DIR,
                                                                    batch_size=32,
                                                                    class_mode='categorical',
                                                                    target_size=(224, 224))
    return train_generator, validation_generator

In [7]:
TRAINING_DIR = "D:\\train\\training\\"
VALIDATION_DIR = "D:\\train\\validation\\"
train_generator, validation_generator = train_val_generators(TRAINING_DIR, VALIDATION_DIR)
train_generator, test_generator = train_val_generators(TRAINING_DIR, VALIDATION_DIR)

Found 1111 images belonging to 14 classes.
Found 288 images belonging to 14 classes.
Found 1111 images belonging to 14 classes.
Found 288 images belonging to 14 classes.


## Model Train

In [ ]:
!pip install tensorflow-hub

In [9]:
import tensorflow_hub as hub
URL = "https://tfhub.dev/google/imagenet/mobilenet_v3_small_100_224/classification/5"
feature_extractor = hub.KerasLayer(URL,input_shape=(224, 224,3),trainable=False)

### Callback

In [10]:
class CustomEarlyStopping(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        if logs['accuracy'] >= 0.92 and logs['val_accuracy'] >= 0.86:
            print("Training stopped as desired accuracy achieved.")
            self.model.stop_training = True

callbacks = CustomEarlyStopping()

callback_model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath="best_model.h5",
    monitor="val_accuracy",
    save_best_only=True,
    save_weights_only=False,
    mode="max",
    verbose=1
)

In [11]:
def create_model():
    model = tf.keras.models.Sequential([
        feature_extractor,
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(14, activation='softmax')
    ])
  
    model.compile(loss='categorical_crossentropy',
                  optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                  metrics=['accuracy'])

    return model


In [12]:
model = create_model()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1001)              2555993   
                                                                 
 dense (Dense)               (None, 512)               513024    
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                                 
 dense_2 (Dense)             (None, 128)               32896     
                                                                 
 batch_normalization (BatchN  (None, 128)              512       
 ormalization)                                                   
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                        

In [13]:
model = create_model()

history = model.fit(train_generator,
                    epochs=20,
                    verbose=1,
                    validation_data=validation_generator,
                    callbacks=[callbacks, callback_model_checkpoint])


Epoch 1/20
35/35 [==============================] - ETA: 0s - loss: 1.9084 - accuracy: 0.4338
Epoch 1: val_accuracy improved from -inf to 0.63194, saving model to best_model.h5
35/35 [==============================] - 31s 724ms/step - loss: 1.9084 - accuracy: 0.4338 - val_loss: 1.0853 - val_accuracy: 0.6319
Epoch 2/20
35/35 [==============================] - ETA: 0s - loss: 0.9179 - accuracy: 0.6976
Epoch 2: val_accuracy improved from 0.63194 to 0.75347, saving model to best_model.h5
35/35 [==============================] - 23s 656ms/step - loss: 0.9179 - accuracy: 0.6976 - val_loss: 0.7312 - val_accuracy: 0.7535
Epoch 3/20
35/35 [==============================] - ETA: 0s - loss: 0.7063 - accuracy: 0.7723
Epoch 3: val_accuracy improved from 0.75347 to 0.81250, saving model to best_model.h5
35/35 [==============================] - 23s 648ms/step - loss: 0.7063 - accuracy: 0.7723 - val_loss: 0.5370 - val_accuracy: 0.8125
Epoch 4/20
35/35 [==============================] - ETA: 0s - loss:

In [14]:
model_results_1 = model.evaluate(validation_generator)

9/9 [==============================] - 3s 278ms/step - loss: 0.4749 - accuracy: 0.8611


## Test


In [15]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.utils import load_img, img_to_array
from PIL import Image

image_path = input("Enter the image path: ")

# Display the selected image path
print("Selected image path:", image_path)

# Load and preprocess the image
img = load_img(image_path, target_size=(224, 224))
x = img_to_array(img)
x /= 255
x = np.expand_dims(x, axis=0)
images = np.vstack([x])

# Predict the image class
classes = model.predict(images, batch_size=1)
label_name = list(train_generator.class_indices.keys())[np.argmax(classes)]
print(f"Predicted probabilities: {classes}")
print(f"Predicted category: {np.argmax(classes)}")
print(f"Predicted label: {label_name}")

Enter the image path: D:\images.jpg
Selected image path: D:\images.jpg
1/1 [==============================] - 1s 593ms/step
Predicted probabilities: [[1.5082088e-01 1.6591432e-02 1.8410555e-03 3.1756057e-04 1.2060605e-03
  5.9467419e-03 2.2087179e-03 1.6413109e-03 2.4344882e-03 7.3356485e-01
  1.6255883e-02 1.7298721e-02 8.5451445e-03 4.1327175e-02]]
Predicted category: 9
Predicted label: tahu_goreng


## Save
.h5

In [16]:
import os
from tensorflow.keras.models import save_model

# Define the file path for the H5 file
model_path = 'D:\\model.h5'

# Check if the file exists and delete it if it does
if os.path.isfile(model_path):
 os.remove(model_path)

# Save the model
save_model(model, model_path)

.pb

In [17]:
# Save the model in the .pb format
saved_model_dir = "saved"
tf.saved_model.save(model, saved_model_dir)


INFO:tensorflow:Assets written to: saved\assets


INFO:tensorflow:Assets written to: saved\assets


In [ ]:
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Model

# Ganti 'model' dengan objek model yang sesuai
# model = ...

# Visualisasi arsitektur model
plot_model(model, to_file='model_architecture.png', show_shapes=True)
